In [1]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from approach.ResEmoteNet import ResEmoteNet

# Select the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the trained model
model = ResEmoteNet().to(device)
checkpoint = torch.load('./Weights/fer_model.pth', map_location=device) # YOU HAVE TO DOWNLOAD AND PUT THE WEIGHT FILE OF THE FER MODEL HERE

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Define a dummy input for ONNX export
dummy_input = torch.randn(1, 3, 64, 64, device=device)  # Batch size 1, 3 color channels, 64x64 resolution

# Specify the output ONNX file path
onnx_file_path = "ResEmoteNet.onnx"

# Export the model to ONNX format
torch.onnx.export(
    model,                        # Model to export
    dummy_input,                  # Dummy input for tracing
    onnx_file_path,               # Path to save the ONNX file
    export_params=True,           # Store trained parameter weights inside the model
    opset_version=11,             # ONNX version to export to
    do_constant_folding=True,     # Optimize the model by folding constant nodes
    input_names=['input'],        # Name of the input tensor
    output_names=['output'],      # Name of the output tensor
    dynamic_axes={                # Allow for variable input sizes
        'input': {0: 'batch_size'},  # Dynamic batch size
        'output': {0: 'batch_size'}
    }
)

print(f"Model has been successfully exported to {onnx_file_path}")


Using device: cuda
Model has been successfully exported to /home/theosiam/Repos/Autotrust/Autotrust/Facial_Expression_Recognition/ResEmoteNet.onnx
